# EchoPro Bootstrapping Workflow <a class="tocSkip">

# Import libraries and configure the Jupyter notebook

In [ ]:
# libraries used in the Notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

# Python version of EchoPro
import EchoPro

# Allows us to grab the SemiVariogram class so we can use its models
from EchoPro.computation import SemiVariogram as SV

# Allows us to easily use matplotlib widgets in our Notebook
%matplotlib widget
# %matplotlib inline

In [ ]:
%%time
survey_2019 = EchoPro.Survey(init_file_path='../config_files/initialization_config.yml',
                             survey_year_file_path='../config_files/survey_year_2019_config.yml',
                             source=3, 
                             exclude_age1=True)

In [ ]:
%%time 
survey_2019.load_survey_data() 

# Get bootstrapping object
- Obtain a bootstrapping object tied to a specific survey year

In [ ]:
boot_obj = survey_2019.get_bootstrapping()

# Run bootstrapping without Kriging
In EchoPro we can perform bootstrapping on a reduced number of transects and obtain the areal biomass density for each iteration of bootstrapping as well as the associated mean Jolly-Hampton CV value. 

In [ ]:
%%time
boot_results_no_krig = boot_obj.run_bootstrapping(run_kriging=False,
                                                  removal_percentage=50.0,
                                                  num_iterations=5,
                                                  seed=1234)

In [ ]:
boot_results_no_krig

In [ ]:
print(f"Mean total biomass without Kriging: {1e-6*boot_results_no_krig['tot_biomass_no_kriging'].mean()} kmt")

# Run bootstrapping with Kriging
Now, we demonstrate how to run bootstrapping with Kriging 

## Initialize Kriging Mesh
Before we run bootstrapping we need to establish what Kriging mesh we will be using. Note: In bootstrapping the transformation of all meshes will be done automatically.   

In [ ]:
krig_mesh = survey_2019.get_kriging_mesh()

## Set appropriate bootstrap Kriging parameters and run boostrapping 

In [ ]:
bootstrap_kriging_params = dict(
    # kriging parameters
    k_max=10,
    k_min=3,
    R=0.0226287,
    ratio=0.001,
    
    # parameters for semi-variogram model
    s_v_params={'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
                'exp_pow': 1.5, 'ls_hole_eff': 0.0},
    
    # grab appropriate semi-variogram model
    s_v_model=SV.generalized_exp_bessel,
    
    # set Kriging mesh
    krig_mesh=krig_mesh 
)

In [ ]:
%%time
boot_results_krig = boot_obj.run_bootstrapping(run_kriging=True,
                                               kriging_params=bootstrap_kriging_params,
                                               removal_percentage=50.0,
                                               num_iterations=5,
                                               seed=1234)

In [ ]:
boot_results_krig

In [ ]:
print(f"Mean total biomass with Kriging: {1e-6*boot_results_krig['tot_biomass_kriging'].mean()} kmt")